# OpenAI Tool Calling Agent - Dummy Weather Example

* Collaborator: Roberto Rodriguez @Cyb3rWard0g

## Import Environment Variables

## Enable Logging

In [10]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

True

In [11]:
import logging

logging.basicConfig(level=logging.INFO)

## Define Tools

In [12]:
from dapr_agents import tool
from pydantic import BaseModel, Field

class GetWeatherSchema(BaseModel):
    location: str = Field(description="location to get weather for")

@tool(args_model=GetWeatherSchema)
def get_weather(location: str) -> str:
    """Get weather information for a specific location."""
    import random
    temperature = random.randint(60, 80)
    return f"{location}: {temperature}F."

class JumpSchema(BaseModel):
    distance: str = Field(description="Distance for agent to jump")

@tool(args_model=JumpSchema)
def jump(distance: str) -> str:
    """Jump a specific distance."""
    return f"I jumped the following distance {distance}"

tools = [get_weather,jump]

## Initialize Agent

In [13]:
from dapr_agents import Agent

AIAgent = Agent(
    name="Rob",
    role= "Weather Assistant",
    tools=tools
)

INFO:dapr_agents.llm.openai.client.base:Initializing OpenAI client...
INFO:dapr_agents.tool.executor:Tool registered: GetWeather
INFO:dapr_agents.tool.executor:Tool registered: Jump
INFO:dapr_agents.tool.executor:Tool Executor initialized with 2 tool(s).
INFO:dapr_agents.agents.base:Constructing system_prompt from agent attributes.
INFO:dapr_agents.agents.base:Using system_prompt to create the prompt template.
INFO:dapr_agents.agents.base:Pre-filled prompt template with attributes: ['name', 'role', 'goal']


In [14]:
AIAgent.prompt_template

ChatPromptTemplate(input_variables=['chat_history'], pre_filled_variables={'name': 'Rob', 'role': 'Weather Assistant', 'goal': 'Help humans'}, messages=[('system', "# Today's date is: June 30, 2025\n\n## Name\nYour name is {{name}}.\n\n## Role\nYour role is {{role}}.\n\n## Goal\n{{goal}}."), MessagePlaceHolder(variable_name=chat_history)], template_format='jinja2')

In [15]:
AIAgent.tools[0].name

'GetWeather'

## Run Agent

In [16]:
await AIAgent.run("Hi my name is Roberto")

INFO:dapr_agents.agents.agent.agent:Iteration 1/10 started.
INFO:dapr_agents.llm.utils.request:Tools are available in the request.
INFO:dapr_agents.llm.openai.chat:Invoking ChatCompletion API.


user:
Hi my name is Roberto

--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.


assistant:
{'content': 'Hello Roberto! How can I assist you with the weather today?', 'role': 'assistant'}

--------------------------------------------------------------------------------



'Hello Roberto! How can I assist you with the weather today?'

In [17]:
AIAgent.chat_history

[MessageContent(content='Hi my name is Roberto', role='user'),
 MessageContent(content='Hello Roberto! How can I assist you with the weather today?', role='assistant')]

In [18]:
await AIAgent.run("What is the weather in Virgina, New York and Washington DC?")

INFO:dapr_agents.agents.agent.agent:Iteration 1/10 started.
INFO:dapr_agents.llm.utils.request:Tools are available in the request.
INFO:dapr_agents.llm.openai.chat:Invoking ChatCompletion API.


user:
What is the weather in Virgina, New York and Washington DC?

--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.
INFO:dapr_agents.agents.agent.agent:Executing GetWeather with arguments {'location': 'Virginia'}
INFO:dapr_agents.tool.executor:Running tool (auto): GetWeather
INFO:dapr_agents.agents.agent.agent:Executing GetWeather with arguments {'location': 'New York'}
INFO:dapr_agents.tool.executor:Running tool (auto): GetWeather
INFO:dapr_agents.agents.agent.agent:Executing GetWeather with arguments {'location': 'Washington DC'}
INFO:dapr_agents.tool.executor:Running tool (auto): GetWeather
INFO:dapr_agents.agents.agent.agent:Iteration 2/10 started.
INFO:dapr_agents.llm.utils.request:Tools are available in the request.
INFO:dapr_agents.llm.openai.chat:Invoking ChatCompletion API.


assistant:
{'content': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_laqow4gKWO2mQVGxTlEAgBl8', 'type': 'function', 'function': {'name': 'GetWeather', 'arguments': '{"location": "Virginia"}'}}, {'id': 'call_ZKm6RcAtrS4EHIWZhOofb0eK', 'type': 'function', 'function': {'name': 'GetWeather', 'arguments': '{"location": "New York"}'}}, {'id': 'call_RLzaaec0MV3khLo1ExLVCw9b', 'type': 'function', 'function': {'name': 'GetWeather', 'arguments': '{"location": "Washington DC"}'}}]}

--------------------------------------------------------------------------------

GetWeather(tool) (Id: call_laqow4gKWO2mQVGxTlEAgBl8):
Virginia: 63F.

--------------------------------------------------------------------------------

GetWeather(tool) (Id: call_ZKm6RcAtrS4EHIWZhOofb0eK):
New York: 66F.

--------------------------------------------------------------------------------

GetWeather(tool) (Id: call_RLzaaec0MV3khLo1ExLVCw9b):
Washington DC: 73F.

-----------------------------------------------------

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.


assistant:
{'content': "Here's the current weather in the requested locations:\n- Virginia: 63°F\n- New York: 66°F\n- Washington DC: 73°F\n\nLet me know if there's anything else I can help you with!", 'role': 'assistant'}

--------------------------------------------------------------------------------



"Here's the current weather in the requested locations:\n- Virginia: 63°F\n- New York: 66°F\n- Washington DC: 73°F\n\nLet me know if there's anything else I can help you with!"